In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt


In [ ]:
class Discontinuous_Galerkin_Solver(object):
    def __init__(self):
        self.xl, self.xr = xl, xr
        self.T = T
        self.M, self.N = M, N
        self.dx = (xr - xl) / M
        self.dt = T / N
        self.x = np.arange(self.xl + self.dx, self.xr + self.dx, self.dx).reshape(
            (1, M)
        )

        self.kn = kn


    def scale_flux(self, kn):
        return kn


    def get_block_matrix(self):
        mu = self.dt / self.dx / 2.0
        sigma_1 = self.scale_flux(self.kn) / self.kn * mu
        sigma_2 = self.scale_flux(self.kn) * self.kn * mu

        A = (
            np.diag((1.0 + 2.0 * sigma_1) * np.ones(M))
            + np.diag(-sigma_1 * np.ones(M - 1), 1)
            + np.diag(-sigma_1 * np.ones(M - 1), -1)
        )
        A[0, -1] = -sigma_1
        A[-1, 0] = -sigma_1

        B = np.diag(mu * np.ones(M - 1), 1) + np.diag(-mu * np.ones(M - 1), -1)
        B[0, -1] = -mu
        B[-1, 0] = mu

        C = (
            np.diag((kn**2 + 2.0 * sigma_2 + self.dt) * np.ones(M))
            + np.diag(-sigma_2 * np.ones(M - 1), 1)
            + np.diag(-sigma_2 * np.ones(M - 1), -1)
        )
        C[0, -1] = -sigma_2
        C[-1, 0] = -sigma_2

        return np.hstack((np.vstack((A, B)), np.vstack((B, C))))


    def exact_f(self, x, v, t):
        r = -2.0 / (1.0 + np.sqrt(1.0 - 4.0 * self.kn**2))
        f = (np.sin(x) / r + kn * v * np.cos(x)) * np.exp(r * t)

        return f


    def decomposition(self, x, t):
        alpha = self.exact_f(x=x, v=1.0, t=t)
        beta = self.exact_f(x=x, v=-1.0, t=t)
        r = (alpha + beta) / 2.0
        j = (alpha - beta) / self.kn / 2.0

        return r, j


    def Jacobi_iteration(self, matrix, vector, iter=100):
        d = np.diag(matrix)
        v = vector / d.reshape(-1, 1)
        iter_matrix = np.matmul(np.diag(1.0 / d), np.diag(d) - matrix)

        x_init = np.zeros(vector.shape)
        for it in range(iter):
            x_init = np.matmul(iter_matrix, x_init) + v

        return x_init


    def initialize_sol(self):
        r0, j0 = self.decomposition(x=self.x, t=0)

        R, J = np.zeros((self.N, self.M)), np.zeros((self.N, self.M))
        R[0:1, :] = r0
        J[0:1, :] = j0

        S = np.hstack((R, J))

        return S


    def solve(self, Iter=100):
        S = self.initialize_sol()
        Block_matrix = self.get_block_matrix()
        for n in range(self.N - 1):
            current_vector = np.hstack(
                (S[n, 0 : self.M], self.kn**2 * S[n, self.M :])
            ).reshape(-1, 1)
            S[n + 1, :] = self.Jacobi_iteration(
                matrix=Block_matrix, vector=current_vector, iter=Iter
            ).reshape(
                -1,
            )

        R_num, J_num = S[-1, 0:M], S[-1, M:]

        return R_num, J_num


    def get_solutions(self):
        sols = {}
        R_num, J_num = self.solve(Iter=100)
        alpha_num = R_num + self.kn * J_num
        beta_num = R_num - kn * J_num
        sols.update({"num": (alpha_num, beta_num)})
        R_ex, J_ex = self.decomposition(x=self.x, t=self.T)
        alpha_ex = R_ex + self.kn * J_ex
        beta_ex = R_ex - self.kn * J_ex
        sols.update({"ex": (alpha_ex, beta_ex)})

        return sols


In [ ]:
kn = 0.5
xl, xr = -np.pi, np.pi
T = 1.0
M_list = [32, 64, 128, 256]
err_list = []
for M in M_list:
    N = M
    solver = Discontinuous_Galerkin_Solver()
    solutions = solver.get_solutions()
    alpha_num, beta_num = solutions["num"]
    alpha_ex, beta_ex = solutions["ex"]
    err = np.max(np.abs(alpha_ex - alpha_num))
    err_list.append(err)
    print(err)


In [ ]:
h_list = [(xr-xl)/M for M in M_list]
plt.figure(figsize=(6, 4))
plt.loglog(h_list, err_list, "rx", label="order")
plt.loglog(h_list, h_list, "-.", label="ref line (slope = 1)")
plt.xlabel("h")
plt.ylabel("error")
plt.legend()
plt.grid()
plt.savefig("./order_test_ex1.png")
plt.show()


In [ ]:
kn = 1e-6
xl, xr = -np.pi, np.pi
T = 0.1
M_list = [32, 64, 128, 256]
err_list = []
for M in M_list:
    N = M
    solver = Discontinuous_Galerkin_Solver()
    solutions = solver.get_solutions()
    alpha_num, beta_num = solutions["num"]
    alpha_ex, beta_ex = solutions["ex"]
    err = np.max(np.abs(alpha_ex - alpha_num))
    err_list.append(err)
    print(err)


In [ ]:
h_list = [(xr-xl)/M for M in M_list]
plt.figure(figsize=(6, 4))
plt.loglog(h_list, err_list, "rx", label="order")
plt.loglog(h_list, h_list, "-.", label="ref line (slope = 1)")
plt.xlabel("h")
plt.ylabel("error")
plt.legend()
plt.grid()
plt.savefig("./order_test_ex2.png")
# plt.show()
